<a href="https://colab.research.google.com/github/0LucasMatheus/google-colab/blob/main/engenharia_de_atributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ATIVIDADE
Aplicar conceitos de feature engineering e avaliar como novas variáveis podem melhorar o desempenho de um modelo de Random Forest na predição de sobrevivência de passageiros do Titanic, utilizando os datasets "/content/train.csv" e "/content/test.csv". Implementar um modelo básico de Random Forest com as variáveis originais (`Pclass`, `Sex`, `Age`, `Fare`) como Modelo A (baseline), criar as features `FamilySize` (`SibSp + Parch + 1`) e `IsAlone` (`(FamilySize == 1).astype(int)`), e comparar o desempenho do Modelo A com um Modelo B (com as duas novas features) usando validação cruzada (`cross_val_score`).

## Carregar os dados

Carregar os datasets de treino e teste (`train.csv` e `test.csv`) em dataframes pandas.


In [ ]:
import pandas as pd

df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

display(df_train.head())
display(df_test.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Preparar os dados para o modelo A (baseline)

Selecionar as variáveis `Pclass`, `Sex`, `Age`, `Fare` e tratar valores ausentes (se houver). Codificar a variável `Sex`.


In [ ]:
X_train_A = df_train[['Pclass', 'Sex', 'Age', 'Fare']].copy()
y_train_A = df_train['Survived'].copy()
X_test_A = df_test[['Pclass', 'Sex', 'Age', 'Fare']].copy()

# Preencher valores ausentes com a mediana dos dados de treino
median_age_train = X_train_A['Age'].median()
median_fare_train = X_train_A['Fare'].median()

X_train_A.loc[:, 'Age'] = X_train_A['Age'].fillna(median_age_train)
X_test_A.loc[:, 'Age'] = X_test_A['Age'].fillna(median_age_train)
X_train_A.loc[:, 'Fare'] = X_train_A['Fare'].fillna(median_fare_train)
X_test_A.loc[:, 'Fare'] = X_test_A['Fare'].fillna(median_fare_train)

# Codificar a coluna 'Sex'
X_train_A.loc[:, 'Sex'] = X_train_A['Sex'].map({'female': 0, 'male': 1})
X_test_A.loc[:, 'Sex'] = X_test_A['Sex'].map({'female': 0, 'male': 1})

display(X_train_A.head())
display(y_train_A.head())
display(X_test_A.head())

,Pclass,Sex,Age,Fare
0,3,1,22.0,7.2500
1,1,0,38.0,71.2833
2,3,0,26.0,7.9250
3,1,0,35.0,53.1000
4,3,1,35.0,8.0500


,Survived
0,0
1,1
2,1
3,1
4,0


,Pclass,Sex,Age,Fare
0,3,1,34.5,7.8292
1,3,0,47.0,7.0000
2,2,1,62.0,9.6875
3,3,1,27.0,8.6625
4,3,0,22.0,12.2875


## Implementar e avaliar o modelo a (random forest baseline)

Treinar um modelo de Random Forest com as variáveis originais (`Pclass`, `Sex`, `Age`, `Fare`) e avaliar seu desempenho usando validação cruzada (`cross_val_score`).


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Instanciar o modelo
model_A = RandomForestClassifier(random_state=42)

# Realizar validação cruzada
cv_scores_A = cross_val_score(model_A, X_train_A, y_train_A, cv=5, scoring='accuracy')

# Imprimir os resultados
print("Cross-validation scores for Model A (Baseline):", cv_scores_A)
print("Mean cross-validation score for Model A (Baseline):", cv_scores_A.mean())

Cross-validation scores for Model A (Baseline): [0.7877095  0.81460674 0.84269663 0.79775281 0.84831461]
Mean cross-validation score for Model A (Baseline): 0.8182160567447114


## Criar as novas features

Calcular as novas features `FamilySize` e `IsAlone` nos datasets de treino e teste.


In [ ]:
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1

df_train['IsAlone'] = (df_train['FamilySize'] == 1).astype(int)
df_test['IsAlone'] = (df_test['FamilySize'] == 1).astype(int)

display(df_train[['FamilySize', 'IsAlone']].head())
display(df_test[['FamilySize', 'IsAlone']].head())

,FamilySize,IsAlone
0,2,0
1,2,0
2,1,1
3,2,0
4,1,1


,FamilySize,IsAlone
0,1,1
1,2,0
2,1,1
3,1,1
4,3,0


In [ ]:
# Preparar os dados para o modelo B (com novas features)

# Selecionar as variáveis para o Modelo B (Pclass, Sex, Age, Fare, FamilySize, IsAlone)
X_train_B = df_train[['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'IsAlone']].copy()
y_train_B = df_train['Survived'].copy()
X_test_B = df_test[['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'IsAlone']].copy()

# Tratar valores ausentes (usando as medianas calculadas anteriormente)
X_train_B.loc[:, 'Age'] = X_train_B['Age'].fillna(median_age_train)
X_test_B.loc[:, 'Age'] = X_test_B['Age'].fillna(median_age_train)
X_train_B.loc[:, 'Fare'] = X_train_B['Fare'].fillna(median_fare_train)
X_test_B.loc[:, 'Fare'] = X_test_B['Fare'].fillna(median_fare_train)

# Codificar a variável 'Sex'
X_train_B.loc[:, 'Sex'] = X_train_B['Sex'].map({'female': 0, 'male': 1})
X_test_B.loc[:, 'Sex'] = X_test_B['Sex'].map({'female': 0, 'male': 1})

display(X_train_B.head())
display(y_train_B.head())
display(X_test_B.head())

# Implementar e avaliar o modelo B (random forest com novas features)

# Instanciar o modelo
model_B = RandomForestClassifier(random_state=42)

# Realizar validação cruzada
cv_scores_B = cross_val_score(model_B, X_train_B, y_train_B, cv=5, scoring='accuracy')

# Imprimir os resultados
print("\nCross-validation scores for Model B (com novas features):", cv_scores_B)
print("Mean cross-validation score for Model B (com novas features):", cv_scores_B.mean())

,Pclass,Sex,Age,Fare,FamilySize,IsAlone
0,3,1,22.0,7.2500,2,0
1,1,0,38.0,71.2833,2,0
2,3,0,26.0,7.9250,1,1
3,1,0,35.0,53.1000,2,0
4,3,1,35.0,8.0500,1,1


,Survived
0,0
1,1
2,1
3,1
4,0


,Pclass,Sex,Age,Fare,FamilySize,IsAlone
0,3,1,34.5,7.8292,1,1
1,3,0,47.0,7.0000,2,0
2,2,1,62.0,9.6875,1,1
3,3,1,27.0,8.6625,1,1
4,3,0,22.0,12.2875,3,0



Cross-validation scores for Model B (com novas features): [0.76536313 0.80337079 0.85393258 0.79213483 0.84831461]
Mean cross-validation score for Model B (com novas features): 0.8126231874960768


## Comparar o desempenho dos modelos


In [ ]:
mean_score_A = cv_scores_A.mean()
mean_score_B = cv_scores_B.mean()

print("\nComparação de Desempenho dos Modelos:")
print(f"Acurácia média de validação cruzada para o Modelo A (Baseline): {mean_score_A:.4f}")
print(f"Acurácia média de validação cruzada para o Modelo B (Original + FamilySize, IsAlone): {mean_score_B:.4f}")

if mean_score_B > mean_score_A:
    print("\nAs novas features (FamilySize e IsAlone) melhoraram o desempenho do modelo.")
elif mean_score_B < mean_score_A:
    print("\nAs novas features (FamilySize e IsAlone) diminuíram ligeiramente o desempenho do modelo.")
else:
    print("\nAs novas features (FamilySize e IsAlone) não alteraram significativamente o desempenho do modelo.")


Comparação de Desempenho dos Modelos:
Acurácia média de validação cruzada para o Modelo A (Baseline): 0.8182
Acurácia média de validação cruzada para o Modelo B (Original + FamilySize, IsAlone): 0.8126

As novas features (FamilySize e IsAlone) diminuíram ligeiramente o desempenho do modelo.


## Criar novas features



In [ ]:
# 1. Criar feature 'Title'
df_train['Title'] = df_train['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
df_test['Title'] = df_test['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)

# Agrupar títulos raros
rare_titles = ['Dr', 'Rev', 'Col', 'Major', 'Capt', 'Sir', 'Lady', 'Countess', 'Jonkheer', 'Dona']
df_train['Title'] = df_train['Title'].replace(rare_titles, 'Rare')
df_test['Title'] = df_test['Title'].replace(rare_titles, 'Rare')

# Mapear títulos para valores numéricos
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
df_train['Title'] = df_train['Title'].map(title_mapping)
df_test['Title'] = df_test['Title'].map(title_mapping)

# Preencher valores ausentes de 'Title' com um placeholder (ex: 0)
df_train['Title'] = df_train['Title'].fillna(0)
df_test['Title'] = df_test['Title'].fillna(0)


# 2. Criar feature 'Embarked_Encoded'
# Preencher valores ausentes de 'Embarked' com o valor mais frequente
most_frequent_embarked = df_train['Embarked'].mode()[0]
df_train['Embarked'] = df_train['Embarked'].fillna(most_frequent_embarked)
df_test['Embarked'] = df_test['Embarked'].fillna(most_frequent_embarked)

# Codificar a coluna 'Embarked'
embarked_mapping = {'S': 0, 'C': 1, 'Q': 2}
df_train['Embarked_Encoded'] = df_train['Embarked'].map(embarked_mapping)
df_test['Embarked_Encoded'] = df_test['Embarked'].map(embarked_mapping)


# 3. Criar feature 'Fare_per_Person'
# Preencher valores ausentes de 'Fare' com a mediana calculada anteriormente
df_train['Fare'] = df_train['Fare'].fillna(median_fare_train)
df_test['Fare'] = df_test['Fare'].fillna(median_fare_train)

df_train['Fare_per_Person'] = df_train['Fare'] / df_train['FamilySize']
df_test['Fare_per_Person'] = df_test['Fare'] / df_test['FamilySize']

# Exibir os dataframes com as novas features
display(df_train[['Title', 'Embarked_Encoded', 'Fare_per_Person']].head())
display(df_test[['Title', 'Embarked_Encoded', 'Fare_per_Person']].head())

,Title,Embarked_Encoded,Fare_per_Person
0,1.0,0,3.62500
1,3.0,1,35.64165
2,2.0,0,7.92500
3,3.0,0,26.55000
4,1.0,0,8.05000


,Title,Embarked_Encoded,Fare_per_Person
0,1.0,2,7.829200
1,3.0,0,3.500000
2,1.0,2,9.687500
3,1.0,0,8.662500
4,3.0,0,4.095833


## Preparar os dados para o modelo c (com todas as features)

Selecionar todas as features (originais, `FamilySize`, `IsAlone` e as três novas) e tratar valores ausentes. Codificar variáveis categóricas, se necessário.


In [ ]:
X_train_C = df_train[['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'IsAlone', 'Title', 'Embarked_Encoded', 'Fare_per_Person']].copy()
y_train_C = df_train['Survived'].copy()
X_test_C = df_test[['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'IsAlone', 'Title', 'Embarked_Encoded', 'Fare_per_Person']].copy()

# Preencher quaisquer valores ausentes restantes usando as medianas calculadas anteriormente
X_train_C.loc[:, 'Age'] = X_train_C['Age'].fillna(median_age_train)
X_test_C.loc[:, 'Age'] = X_test_C['Age'].fillna(median_age_train)
X_train_C.loc[:, 'Fare'] = X_train_C['Fare'].fillna(median_fare_train)
X_test_C.loc[:, 'Fare'] = X_test_C['Fare'].fillna(median_fare_train)

# Garantir que a coluna 'Sex' esteja codificada numericamente
X_train_C.loc[:, 'Sex'] = X_train_C['Sex'].map({'female': 0, 'male': 1})
X_test_C.loc[:, 'Sex'] = X_test_C['Sex'].map({'female': 0, 'male': 1})

display(X_train_C.head())
display(y_train_C.head())
display(X_test_C.head())

,Pclass,Sex,Age,Fare,FamilySize,IsAlone,Title,Embarked_Encoded,Fare_per_Person
0,3,1,22.0,7.2500,2,0,1.0,0,3.62500
1,1,0,38.0,71.2833,2,0,3.0,1,35.64165
2,3,0,26.0,7.9250,1,1,2.0,0,7.92500
3,1,0,35.0,53.1000,2,0,3.0,0,26.55000
4,3,1,35.0,8.0500,1,1,1.0,0,8.05000


,Survived
0,0
1,1
2,1
3,1
4,0


,Pclass,Sex,Age,Fare,FamilySize,IsAlone,Title,Embarked_Encoded,Fare_per_Person
0,3,1,34.5,7.8292,1,1,1.0,2,7.829200
1,3,0,47.0,7.0000,2,0,3.0,0,3.500000
2,2,1,62.0,9.6875,1,1,1.0,2,9.687500
3,3,1,27.0,8.6625,1,1,1.0,0,8.662500
4,3,0,22.0,12.2875,3,0,3.0,0,4.095833


## Implementar e avaliar o modelo c



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Instanciar o modelo
model_C = RandomForestClassifier(random_state=42)

# Realizar validação cruzada
cv_scores_C = cross_val_score(model_C, X_train_C, y_train_C, cv=5, scoring='accuracy')

# Imprimir os resultados
print("Cross-validation scores for Model C (com novas features):", cv_scores_C)
print("Mean cross-validation score for Model C (com novas features):", cv_scores_C.mean())

Cross-validation scores for Model C (com novas features): [0.80446927 0.80337079 0.86516854 0.78651685 0.80898876]
Mean cross-validation score for Model C (com novas features): 0.8137028435126483


## Analisar a importância das variáveis


In [ ]:
# Treinar o modelo com todos os dados de treino
model_C.fit(X_train_C, y_train_C)

# Obter a importância das features
feature_importances = pd.Series(model_C.feature_importances_, index=X_train_C.columns)

# Ordenar a importância das features em ordem decrescente
sorted_feature_importances = feature_importances.sort_values(ascending=False)

# Imprimir ou exibir a importância das features ordenadas
print("\nImportância das Features para o Modelo C:")
display(sorted_feature_importances)


Importância das Features para o Modelo C:


,0
Age,0.204756
Sex,0.173724
Fare_per_Person,0.159368
Title,0.154707
Fare,0.148550
Pclass,0.067323
FamilySize,0.057126
Embarked_Encoded,0.025282
IsAlone,0.009164


## Comparar o desempenho dos modelos e discutir as features



In [ ]:
print("\nComparação de Acurácia Média de Validação Cruzada:")
print(f"Modelo A (Baseline): {mean_score_A:.4f}")
print(f"Modelo B (Original + FamilySize, IsAlone): {mean_score_B:.4f}")
print(f"Modelo C (Todas as Features): {cv_scores_C.mean():.4f}")

print("\nAnálise da Comparação:")
if cv_scores_C.mean() > mean_score_B and cv_scores_C.mean() > mean_score_A:
    print("O Modelo C teve desempenho melhor que os Modelos A e B.")
elif cv_scores_C.mean() > mean_score_A and cv_scores_C.mean() <= mean_score_B:
     print("O Modelo C teve desempenho melhor que o Modelo A, mas similar ou ligeiramente pior que o Modelo B.")
elif cv_scores_C.mean() <= mean_score_A and cv_scores_C.mean() > mean_score_B:
     print("O Modelo C teve desempenho pior que o Modelo A, mas melhor que o Modelo B.")
elif cv_scores_C.mean() <= mean_score_A and cv_scores_C.mean() <= mean_score_B:
    print("O Modelo C teve desempenho pior que os Modelos A e B.")
else:
    print("O Modelo C teve desempenho similar aos Modelos A e B.")


Comparação de Acurácia Média de Validação Cruzada:
Modelo A (Baseline): 0.8182
Modelo B (Original + FamilySize, IsAlone): 0.8126
Modelo C (Todas as Features): 0.8137

Análise da Comparação:
O Modelo C teve desempenho pior que o Modelo A, mas melhor que o Modelo B.


## Análise da Importância das Features e Discussão sobre a Utilidade

Com base na análise de importância das features do Modelo C, as variáveis mais influentes foram 'Age', 'Sex', 'Fare_per_Person', 'Title' e 'Fare'. As novas features 'Title' e 'Fare_per_Person' parecem ser mais úteis do que 'FamilySize' e 'IsAlone', que apresentaram menores pontuações de importância.

Ao comparar as acurácias médias de validação cruzada, o Modelo C (com todas as features) obteve uma acurácia média de {{cv_scores_C.mean():.4f}}. Este valor é ligeiramente inferior ao do Modelo A ({{mean_score_A:.4f}}) e ligeiramente superior ao do Modelo B ({{mean_score_B:.4f}}).

Embora algumas das novas features (como 'Title' e 'Fare_per_Person') tenham mostrado importância relativamente alta individualmente, sua inclusão juntamente com 'FamilySize' e 'IsAlone' no Modelo C não resultou em uma melhoria na acurácia média geral de validação cruzada em comparação com os Modelos A ou B. Isso sugere que as novas features podem não ter capturado informação preditiva adicional suficiente para compensar a potencial complexidade adicionada, ou talvez as interações entre as features sejam importantes e não foram totalmente capturadas pela simples adição destas features.

## Resumo das Conclusões

*   Três novas features ('Title', 'Embarked_Encoded' e 'Fare_per_Person') foram criadas e adicionadas aos conjuntos de dados de treino e teste.
*   O Modelo C, treinado com todas as features, alcançou uma acurácia média de validação cruzada de aproximadamente {{cv_scores_C.mean():.4f}}.
*   Esta acurácia média para o Modelo C foi ligeiramente menor que a do Modelo A (Baseline), que foi de {{mean_score_A:.4f}}, e ligeiramente melhor que a do Modelo B (Original + FamilySize, IsAlone), que foi de {{mean_score_B:.4f}}.
*   A análise de importância das features para o Modelo C mostrou que 'Age', 'Sex', 'Fare_per_Person', 'Title' e 'Fare' foram as features mais importantes em ordem decrescente.
*   Entre as features recém-criadas, 'Title' e 'Fare_per_Person' demonstraram maior importância do que 'FamilySize', 'IsAlone' e 'Embarked_Encoded'.
*   Apesar de algumas novas features mostrarem importância individual, seu efeito combinado no Modelo C não melhorou o desempenho preditivo geral do modelo nesta configuração de validação cruzada. Isso sugere a necessidade de explorar mais a fundo as interações das features ou métodos de codificação alternativos.

## Respostas às Perguntas

### Qual foi a acurácia média antes e depois das novas features?

In [ ]:
print(f"Acurácia média do Modelo A (Baseline): {mean_score_A:.4f}")
print(f"Acurácia média do Modelo B (Original + FamilySize, IsAlone): {mean_score_B:.4f}")
print(f"Acurácia média do Modelo C (Todas as Features): {cv_scores_C.mean():.4f}")

Acurácia média do Modelo A (Baseline): 0.8182
Acurácia média do Modelo B (Original + FamilySize, IsAlone): 0.8126
Acurácia média do Modelo C (Todas as Features): 0.8137


### Quais variáveis foram mais importantes segundo o modelo?

Segundo a análise de importância das features do Modelo C, as variáveis mais importantes foram: 'Age', 'Sex', 'Fare_per_Person', 'Title' e 'Fare'.

### Alguma feature criada teve efeito negativo no desempenho?

A inclusão das features 'FamilySize' e 'IsAlone' no Modelo B resultou em uma acurácia ligeiramente menor em comparação com o Modelo A (Baseline). A inclusão de todas as novas features no Modelo C também resultou em uma acurácia ligeiramente menor que a do Modelo A, mas ligeiramente superior à do Modelo B. Isso sugere que, nesta configuração, as novas features não melhoraram o desempenho geral do modelo.

### Que paralelos podem ser feitos com sua área (ex.: variáveis derivadas em dados biológicos, clínicos ou experimentais)?

Similar ao que foi feito aqui com os dados do Titanic, em dados biológicos ou clínicos, podemos criar variáveis derivadas a partir de dados brutos (ex: calcular IMC a partir de peso e altura, ou criar escores de risco a partir de múltiplos marcadores). A importância e o efeito dessas variáveis derivadas no desempenho de modelos preditivos (ex: predição de resposta a tratamento ou prognóstico de doença) podem variar e precisam ser avaliados, assim como fizemos com as novas features no dataset do Titanic.